In [ ]:
#imports:
import pandas as pd
import numpy as np
import math
from time import time

from sklearn.utils import shuffle

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive 
drive.mount('/mntDrive') 
data=pd.read_csv("/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/input/prepared_facebook_data.csv")

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# grid search
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

param_dist_group_1 = {"max_depth": sp_randint(3, 10),
                  "min_child_weight": sp_randint(1, 10)}

param_dist_group_2 = {"alpha": uniform(loc=0, scale=1),
                  "lambda": uniform(loc=0, scale=1)}

param_dist_group_3 = {"subsample":uniform(loc=0.5, scale=0.4),
                  "colsample_bytree":uniform(loc=0.5, scale=0.4)}


n_iter_search = 243

In [ ]:
# random groups test
# build a classifier
clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)

#first group:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_1,
                                      n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV for first group took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))


res_rand_1=pd.DataFrame(random_search.cv_results_)
res_rand_1['experiment_name']='random search for 1 group'      
res_rand_1['run_number']=1

clf.set_params(**random_search.best_params_)

In [ ]:

#second group:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_2,
                                      n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV for second group took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))

res_rand_2=pd.DataFrame(random_search.cv_results_)
res_rand_2['experiment_name']='random search for 2 group' 
res_rand_2['run_number']=1


clf.set_params(**random_search.best_params_)

In [ ]:
#third group:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_3,
                                      n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV for third group took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))

res_rand_3=pd.DataFrame(random_search.cv_results_)
res_rand_3['experiment_name']='random search for 3 group' 
res_rand_3['run_number']=1

In [ ]:
res_1=pd.concat([res_rand_1,res_rand_2,res_rand_3])
res_1.to_csv("res_groups_1.csv",index=False)

In [ ]:
cp res_groups_1.csv "/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/results/xgboost_perfomance_group_search_GPU_1.csv"